In [8]:
import numpy as np
import tensorflow as tf

import plotly.graph_objects as go

In [9]:
num_quasimomenta = 49
max_momentum_site = 9

max_momentum_site_change = 2 * max_momentum_site
num_momentum_sites = (max_momentum_site_change + 1) ** 2

In [10]:
quasimomentum = np.mgrid[0:2:num_quasimomenta*1j,
                         0:2:num_quasimomenta*1j].reshape(2,-1).T

momentum_site = np.mgrid[-max_momentum_site:max_momentum_site+1:1,
                         -max_momentum_site:max_momentum_site+1:1].reshape(2,-1).T

momentum = 2*momentum_site + quasimomentum[:, np.newaxis]
kinetic = np.linalg.norm(momentum, axis=-1)**2

momentum_site_change = momentum_site[:, np.newaxis] - momentum_site

In [16]:
def render_momentum_site_potential(depth):
    return -1/16 * depth * np.pad(np.array([
        [1, 0, 2, 0, 1],
        [0, 0, 0, 0, 0],
        [2, 0, 4, 0, 2],
        [0, 0, 0, 0, 0],
        [1, 0, 2, 0, 1]
    ]), max_momentum_site_change - 2)

In [17]:
def compute_dispersion(momentum_site_potential):
    potential = momentum_site_potential[
        momentum_site_change[:, :, 0] + max_momentum_site_change,
        momentum_site_change[:, :, 1] + max_momentum_site_change,
    ]

    energy = tf.linalg.diag(kinetic) + potential[np.newaxis, ...]

    dispersion = tf.reshape(tf.linalg.eigvalsh(energy), (num_quasimomenta, num_quasimomenta, num_momentum_sites))

    return dispersion

In [35]:
def compute_tunneling(dispersion):
    tunneling = np.abs(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(dispersion)))) / num_quasimomenta**2
    tunneling[num_quasimomenta//2, num_quasimomenta//2] = np.nan
    return tunneling

In [46]:
momentum_site_potential = render_momentum_site_potential(10.)

spatial_potential = np.real(np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(momentum_site_potential))))

dispersion = compute_dispersion(momentum_site_potential)

ground_band = dispersion[:, :, 0]

tunneling = compute_tunneling(ground_band)
tunneling[num_quasimomenta//2, num_quasimomenta//2] = np.nan

In [47]:
lateral_tunneling = tunneling[num_quasimomenta//2-1, num_quasimomenta//2]
diagonal_tunneling = tunneling[num_quasimomenta//2-1, num_quasimomenta//2-1]

diagonal_tunneling/lateral_tunneling

0.0014958824616390777

In [48]:
figure = go.Figure(data=[go.Surface(z=spatial_potential)])
figure.update_layout(title="Potential", 
                     autosize=False, width=750, height=750)
figure.show()

figure = go.Figure(data=[go.Surface(z=dispersion[:, :, band], showscale=False) for band in range(3)])
figure.update_layout(title="Dispersion",
                     autosize=False, width=750, height=750)
figure.show()

radius = 5
figure = go.Figure(data=[go.Surface(z=tunneling[num_quasimomenta//2-radius:num_quasimomenta//2+radius+1, num_quasimomenta//2-radius:num_quasimomenta//2+radius+1])])
figure.update_layout(title="Tunneling",
                     autosize=False, width=750, height=750)
figure.show()